In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.coordinates import SkyCoord
from astropy.io import fits
from astropy.table import Table, vstack
from astropy.wcs import WCS
from astropy import units as u
from astropy.coordinates import match_coordinates_sky
import os,sys
import pandas as pd
import glob

In [4]:
path='/Users/baotong/data_GalDisc/data/xmmdr14s/'
xmmdr14=fits.open(path+'xmmstack_v3.2_4xmmdr14s.fits.gz')
xmmdr14_obs=fits.open(path+'xmmstack_v3.2_4xmmdr14s_obslist.fits.gz')
allobsnow=np.loadtxt(path+'allobs.txt',dtype='str')

In [5]:
obsdr14=xmmdr14_obs[1].data['OBS_ID']
obsused=np.intersect1d(obsdr14,allobsnow)
print(len(obsused))

562


In [6]:
srclist=xmmdr14[1].data
print(len(np.unique(srclist['SRCID'])))

427524


In [13]:
nobs=srclist['N_CONTRIB']
nobs=nobs[np.where(nobs>0)]
print(np.sort(nobs))

[ 1  1  1 ... 74 75 77]


In [23]:
print(np.sort(srclist['SRCID']))

[3000011010100001 3000011010100001 3000011010100001 ... 3093220090112761
 3093220090114593 3093220090114593]


In [25]:
path='/Users/baotong/data_GalDisc/data/'
galdiscdr14=fits.open(path+'GalDisc_4xmmdr14s.fits')

In [70]:
srclistgal=galdiscdr14[1].data
stackflag=np.array(srclistgal['STACK_FLAG'])
N_OBS=np.array(srclistgal['N_OBS'])
EP_DET_ML=np.array(srclistgal['EP_DET_ML'])
EXTENT_ML=np.array(srclistgal['EXTENT_ML'])
EXTENT=np.array(srclistgal['EXTENT'])
srclistgal_clean=srclistgal[np.where((N_OBS>=1)&(stackflag<=1)&(EP_DET_ML>6)&(EXTENT_ML<0)&(EXTENT==0))]

In [76]:
print(np.where(N_OBS>0))

(array([     0,      8,     15, ..., 122731, 122734, 122737]),)


In [72]:
print(len(srclistgal),len(np.unique(srclistgal['SRCID'])))
print(len(srclistgal_clean),len(np.unique(srclistgal_clean['SRCID'])))
unique_elements, indices = np.unique(srclistgal['SRCID'], return_index=True)
sorted_indices = np.sort(indices)


122740 26542
12986 12986


In [146]:
from astropy.io import fits
import numpy as np

def filter_and_save_indices(fits_file, selected_indices_file, unselected_indices_file):
    with fits.open(fits_file) as hdul:
        data = hdul[1].data
        header = hdul[1].header

        # Initialize lists to hold indices for selected and unselected real sources
        selected_indices = []
        unselected_indices = []

        # Temporary list to collect indices for the current source (real + associated)
        current_group_indices = []
        
        # Iterate over each row in the data
        for i, row in enumerate(data):
            print(i)  # For tracking progress
            if row['N_OBS'] > 0:  # Real source detected
                if current_group_indices:  # If there is already a group, process it
                    # Check if the last source was selected
                    real_source = data[current_group_indices[0]]
                    if (real_source['STACK_FLAG'] <= 1 and 
                        real_source['EP_DET_ML'] >= 6 and 
                        real_source['EXTENT_ML'] <= 0):
                        selected_indices.extend(current_group_indices)
                    else:
                        unselected_indices.extend(current_group_indices)
                
                # Start a new group with the current real source
                current_group_indices = [i]
            else:
                # Add observational index to the current group
                current_group_indices.append(i)

        # Process the last group after the loop finishes
        if current_group_indices:
            real_source = data[current_group_indices[0]]
            if (real_source['STACK_FLAG'] <= 1 and 
                real_source['EP_DET_ML'] >= 6 and 
                real_source['EXTENT_ML'] <= 0):
                selected_indices.extend(current_group_indices)
            else:
                unselected_indices.extend(current_group_indices)

    # Save the indices to text files
    np.savetxt(selected_indices_file, selected_indices, fmt='%d')
    np.savetxt(unselected_indices_file, unselected_indices, fmt='%d')

    print(f"Selected indices saved to {selected_indices_file}")
    print(f"Unselected indices saved to {unselected_indices_file}")

In [152]:
path='/Users/baotong/data_GalDisc/data/'
fits_file=path+'xmmdr14s/'+'GalDisc_4xmmdr14s.fits'
data = fits.open(fits_file)[1].data
selected_indices=np.loadtxt(path+'xmmdr14s/'+'GalDisc_selected_indices.txt',dtype='int')
unselected_indices=np.loadtxt(path+'xmmdr14s/'+'GalDisc_unselected_indices.txt',dtype='int')

In [167]:
select=data[selected_indices]
unselect=data[unselected_indices]
print(len(np.where(select['N_OBS']>0)[0]))
print(len(select))

12986
45994


In [165]:
a=fits.open(fits_file)
print(a['XMMST3.2'].data)

[('4XMMs J171207.5-231428', 3050515010100756, '',          19,           7, 19, 13, 0, 258.03160621, -23.24135759, 1.5613805, 0.63504292,  9.40669546, 921.5001  , 0.15419154, 1226.4775 , 0.35860047,  9.495759, 2, 12, 'TFTFTFTFTT', 'TFTFTFTFT', 'FFTFTFTFF', 'FFTFTFTFF', '', -9223372036854775808, -9223372036854775808,          nan,          nan,       nan,        nan, -32768, -32768, -32768, 2.6603746e-14, 1.3331159e-14, 2.6818514e-16, 6.0629819e-16, 3.6729516e-16, 5.1275171e-16, 3.0954091e-16, 2.6863250e-16, 5.9136732e-15, 1.3519190e-15, 1.1416197e-14, 1.3066641e-14, 2.6177355e-14, 1.4142416e-14, 2.8970118e-16, 8.5543043e-16, 2.7661883e-16, 5.4592952e-16, 2.230271e-16, 2.7016219e-16, 1.4441202e-14, 2.71515734e-15, 1.09468065e-14, 1.3839547e-14, 2.1749055e-14, 5.8376874e-14, 2.0311841e-16, 1.0448552e-15, 1.3105946e-16, 1.91209239e-15, 4.4486789e-15, 3.1009548e-15, 3.0904476e-15, 1.6109601e-15, 1.3875750e-14, 5.8231435e-14, 3.7264689e-14, 5.4750868e-14, 3.3715691e-16, 1.5113106e-15, 2.478

In [161]:
print(len(np.unique(data['OBS_ID'])))

589
